In [3]:
import tensorflow as tf
import numpy as np

In [34]:
def maxpool_with_indices(input, k_size, stride, scope):
    with tf.variable_scope(f"pool{scope}"):
        pooled, indices = tf.nn.max_pool_with_argmax(input, k_size, stride, padding="VALID")
        return pooled, indices


    
#### When think of modifying use : https://www.tensorflow.org/api_docs/python/tf/unravel_index
####                            or https://www.tensorflow.org/api_docs/python/tf/scatter_nd for other dimensions
def unpool(input, indices, upsample_factor, scope):
    with tf.variable_scope(f"unpool{scope}"):
        # input and indices must be of same shape
        new_size = tf.stack([input.shape[0],
                                input.shape[1]*upsample_factor,
                                input.shape[2]*upsample_factor,
                                input.shape[3]])
        batch_num = input.shape[0]
        flattened_size = input.shape[1]*upsample_factor*input.shape[2]*upsample_factor*input.shape[3]
        flat2 = input.shape[1]*input.shape[2]*input.shape[3]
        
        indices = tf.reshape(indices, [input.shape[0], -1])
        batch_indices = [tf.fill([flat2], i) for i in range(batch_num)]
        batch_indices = tf.dtypes.cast(tf.stack(batch_indices), tf.int64)
        print(batch_indices)
        print(indices)
        
        indices = tf.concat([tf.expand_dims(batch_indices, axis=-1), tf.expand_dims(indices, axis=-1)], axis=2)
        print(indices)
        indices = tf.reshape(indices, [-1, 2])
        reshaped_input = tf.reshape(input, [-1])
        print(input)
        
        scatter = tf.scatter_nd(indices, reshaped_input, tf.constant([input.shape[0], flattened_size], tf.int64))
        scatter = tf.reshape(scatter, new_size)
        print(scatter)
        return scatter

In [35]:
batch_size = 2
X = tf.placeholder(tf.float32, (batch_size, 4, 4, 3))

pool, indexes = maxpool_with_indices(X, [1, 2, 2, 1], [1, 2, 2, 1], 1)
unpooled = unpool(pool, indexes, 2, 1)

Tensor("unpool1_12/Cast:0", shape=(2, 12), dtype=int64)
Tensor("unpool1_12/Reshape:0", shape=(2, 12), dtype=int64)
Tensor("unpool1_12/concat:0", shape=(2, 12, 2), dtype=int64)
Tensor("pool1_12/MaxPoolWithArgmax:0", shape=(2, 2, 2, 3), dtype=float32)
Tensor("unpool1_12/Reshape_3:0", shape=(2, 4, 4, 3), dtype=float32)


In [36]:
print(pool)
print(indexes)
print(unpooled)

a = np.random.rand(2, 4, 4, 3)
print(a.shape)

Tensor("pool1_12/MaxPoolWithArgmax:0", shape=(2, 2, 2, 3), dtype=float32)
Tensor("pool1_12/MaxPoolWithArgmax:1", shape=(2, 2, 2, 3), dtype=int64)
Tensor("unpool1_12/Reshape_3:0", shape=(2, 4, 4, 3), dtype=float32)
(2, 4, 4, 3)


In [37]:
with tf.Session() as sess:
    pooling, index, upsampled = sess.run([pool, indexes, unpooled], feed_dict={X: a})

In [38]:
print(a)

[[[[0.38136782 0.18455968 0.86473066]
   [0.32374647 0.40201833 0.84936273]
   [0.59545316 0.56437869 0.87372253]
   [0.75100545 0.76656792 0.92467579]]

  [[0.39207686 0.17686958 0.22115289]
   [0.79461795 0.08157522 0.95699354]
   [0.46016572 0.54912605 0.70999422]
   [0.91211233 0.99661519 0.86192987]]

  [[0.61037157 0.32751105 0.37276562]
   [0.37570618 0.08317121 0.35383032]
   [0.58543629 0.08849293 0.66765511]
   [0.10664542 0.74430346 0.95261446]]

  [[0.9490332  0.12141915 0.52595779]
   [0.0605821  0.50430914 0.52313757]
   [0.24490628 0.21946001 0.73688933]
   [0.20633656 0.0349996  0.99258126]]]


 [[[0.57065046 0.35511431 0.22484371]
   [0.22068036 0.18394342 0.34090796]
   [0.9517712  0.66162852 0.86420947]
   [0.068316   0.99349968 0.77796021]]

  [[0.58827241 0.7360897  0.48052561]
   [0.53078077 0.91134645 0.71696684]
   [0.45503887 0.79169571 0.16588232]
   [0.81299022 0.21411209 0.21377249]]

  [[0.18117532 0.52493451 0.5197181 ]
   [0.53745404 0.36476016 0.18646088

In [39]:
print(pooling)

[[[[0.79461795 0.40201834 0.9569935 ]
   [0.91211236 0.9966152  0.92467576]]

  [[0.9490332  0.5043091  0.52595776]
   [0.5854363  0.74430346 0.99258125]]]


 [[[0.5882724  0.91134644 0.71696687]
   [0.9517712  0.9934997  0.8642095 ]]

  [[0.537454   0.99238694 0.85360837]
   [0.59474915 0.80844045 0.723112  ]]]]


In [40]:
print(index)

[[[[15  4 17]
   [21 22 11]]

  [[36 40 38]
   [30 34 47]]]


 [[[12 16 17]
   [ 6 10  8]]

  [[27 37 41]
   [45 46 35]]]]


In [41]:
print(upsampled)

[[[[0.         0.         0.        ]
   [0.         0.40201834 0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.92467576]]

  [[0.         0.         0.        ]
   [0.79461795 0.         0.9569935 ]
   [0.         0.         0.        ]
   [0.91211236 0.9966152  0.        ]]

  [[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.5854363  0.         0.        ]
   [0.         0.74430346 0.        ]]

  [[0.9490332  0.         0.52595776]
   [0.         0.5043091  0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.99258125]]]


 [[[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.9517712  0.         0.8642095 ]
   [0.         0.9934997  0.        ]]

  [[0.5882724  0.         0.        ]
   [0.         0.91134644 0.71696687]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0.537454   0.         0.        